<a href="https://colab.research.google.com/github/Dimildizio/python_course/blob/main/basics/Functions_part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Python Функции


## Темы:

1. `return` и область видимости (LEGB), локальные/глобальные переменные  
2. Позиционные аргументы и значения по умолчанию (ловушка с изменяемыми)  
3. Именованные (keyword) аргументы и keyword-only  
4. `*args` и `**kwargs`: упаковка и распаковка аргументов  
5. Функции как объекты: передача/возврат, `lambda`, замыкания (`nonlocal`)  



## 1) `return` и область видимости (scope)

**Теория:**
- Функция объявляется через `def`. Если нет `return`, функция возвращает `None`.
- `return` **сразу** завершает функцию и отдаёт значение.
- Области видимости (LEGB):
  - **L**ocal — имена внутри функции,
  - **E**nclosing — внешние области вложенных функций,
  - **G**lobal — имена модуля (файла),
  - **B**uiltins — встроенные имена Python.
- Глобальные переменные менять из функции не стоит. Возвращайте новые значения и присваивайте их снаружи.


#### Пример
считаем урон и золото партии

In [ ]:
def compute_damage(base: int, bonus: int) -> int:
    """Возвращает суммарный урон героя.
    base — урон оружия; bonus — модификатор (например, STR модификатор или бафф).
    """
    total = base + bonus
    return total


# Проверка примера:
assert compute_damage(8, 3) == 11, "compute_damage должен вернуть 11 при (8, 3)"


In [ ]:
party_gold = 100  # глобальное золото пати


def spend_gold_safely(cost: int) -> int:
    """Правильный способ - посчитать новый баланс и вернуть его."""
    new_amount = party_gold - cost   # читаем глобальную переменную (не меняем)
    return new_amount


# Проверка примера:
assert spend_gold_safely(25) == 75, "При party_gold=100 spend_gold_safely(25) должно дать 75"
assert party_gold == 100, "Глобальное золото не должно меняться в примере"


# Плохая идея менять глобальную переменную внутри без необходимости:
def spend_gold_bad(cost: int):
    # global party_gold  # так можно, но лучше избегать
    # party_gold -= cost
    # return party_gold
    pass  # Антипаттерн: лучше возвращать и присваивать снаружи


# Правильно - переопределить, вызвав функци и вернуть результат в переменную
party_gold = spend_gold_safely(25)
assert party_gold == 75, "Глобальное золото должно измениться в примере"

print("Базовые assert пройдены.")


**Задание:**  
Напишите функцию `clamp_hp(current_hp: int, max_hp: int) -> int`, которая возвращает корректные HP героя:  
- если `current_hp < 0` → вернуть `0`,  
- если `current_hp > max_hp` → вернуть `max_hp`,  
- иначе вернуть `current_hp` как есть.

**Требование:** никаких глобальных изменений, только `return`.  
Ниже - тесты `assert`, которые должны проходить.


In [ ]:
# ВАШЕ РЕШЕНИЕ
def clamp_hp(current_hp: int, max_hp: int) -> int:
    # TODO: верните HP в диапазоне [0, max_hp]
    # Подсказка: используйте ранние return
    return None  # <-- временная заглушка

# ТЕСТЫ: раскомментируйте после реализации или оставьте — они покажут, что нужно исправить
assert clamp_hp(-5, 20) == 0
assert clamp_hp(15, 20) == 15
assert clamp_hp(30, 20) == 20
print("Задание 1 — assert пройдены (если вы это видите).")


#### **Разбор/решение:**  


Последовательные проверки и ранний `return` без побочных эффектов.

In [ ]:
def clamp_hp(current_hp: int, max_hp: int) -> int:
  # Также можно использовать min(current_hp, max_hp), max(current_hp, 0)
    if current_hp < 0:
        return 0
    if current_hp > max_hp:
        return max_hp
    return current_hp

assert clamp_hp(-5, 20) == 0
assert clamp_hp(15, 20) == 15
assert clamp_hp(30, 20) == 20
print("Разбор 1 — assert пройдены.")


## 2) Позиционные аргументы и значения по умолчанию

**Теория**
- Позиционные аргументы передаются по порядку: `f(оружие, цель)`.
- Значения по умолчанию: `def f(a, b=10): ...`.
- **Опасность:** изменяемые значения по умолчанию (списки/словари) **сохраняются между вызовами**.
  Шаблон-спаситель:
  ```python
  def f(x, items=None):
      if items is None:
          items = []
  ```
  Пояснение: если бы мы передавали список - то, поскольку он изменяем и определен\создан в момент определния функции (а не вызова), то он един для всех вызовов функции, таким образом лучше использовать шаблон с None где изменяемый тип определяется уже только в локлаьной области видимости


In [ ]:
def cast_spell(spell: str, target: str, level: int = 1) -> str:
    return f"{spell} (ур. {level}) попадает по {target}!"

assert cast_spell("Magic Missile", "гоблину") == "Magic Missile (ур. 1) попадает по гоблину!"
assert cast_spell("Fireball", "оркам", 3) == "Fireball (ур. 3) попадает по оркам!"

In [ ]:
# Ловушка: сумка лута как значение по умолчанию (АНТИПАТТЕРН)
def add_loot(item: str, bag=[]):  # НЕ РЕКОМЕНДУЕТСЯ
    bag.append(item)
    return bag

loot_a = add_loot("зелье лечения")
loot_b = add_loot("серебряная монета")
assert loot_a is loot_b, "Список по умолчанию один и тот же — это ловушка!"
assert loot_b == ["зелье лечения", "серебряная монета"]

# Правильно:
def add_loot_safe(item: str, bag=None):
    if bag is None:
        bag = []
    bag.append(item)
    return bag

safe1 = add_loot_safe("тропа эльфов")
safe2 = add_loot_safe("кольцо защиты")
assert safe1 == ["тропа эльфов"] and safe2 == ["кольцо защиты"]

print("раздела 2 assert пройдены.")


**Задание:**  
Напишите `def battle_cry(phrase: str, times: int = 3, sep: str = ' / ') -> str`,  
которая повторяет боевой клич героя заданное число раз, соединяя через `sep`.

Ниже — тесты `assert`.


In [ ]:
# ВАШЕ РЕШЕНИЕ
def battle_cry() -> str:
    # TODO: верните строку с повторяющимся боевым кличем
    return ""  # заглушка

# ТЕСТЫ
assert battle_ry := battle_cry("За Циридана", 2, " | ") == "За Циридана | За Циридана"
assert battle_cry("В атаку!") == "В атаку! / В атаку! / В атаку!"
print("Задание 2 assert пройдены")


#### **Разбор/решение:**  


Удобно использовать `sep.join([phrase] * times)`.

In [ ]:
def battle_cry(phrase: str, times: int = 3, sep: str = ' / ') -> str:
    return sep.join([phrase] * times)

assert battle_cry("За Лордерон", 2, " | ") == "За Лордерон | За Лордерон"
assert battle_cry("В атаку!") == "В атаку! / В атаку! / В атаку!"
print("Разбор 2 — assert пройдены.")


## 3) Именованные (keyword) аргументы и keyword-only

**Теория**
- Именованные аргументы повышают читаемость: `create_character(name="Lia", race="Elf", cls="Rogue")`.
- Порядок вызова: сначала позиционные(!), затем именованные (могут быть в разном порядке).
- **Keyword-only** параметры (только по имени) объявляются после `*` в сигнатуре.


In [ ]:
Создание персонажа и атака
def create_character(name: str, *, race: str, cls: str, level: int = 1) -> str:
    return f"{name} - {race} {cls}, ур. {level}"

assert create_character("Lia", race="Elf", cls="Rogue", level=3) == "Lia - Elf Rogue, ур. 3"

In [ ]:

def attack(name: str, target: str, *, advantage: bool = False, bonus: int = 0) -> str:
    note = " с преимуществом" if advantage else ""
    return f"{name} атакует {target}{note} (бонус {bonus})."

assert attack("Borin", "тролля", advantage=True, bonus=2) == "Borin атакует тролля с преимуществом (бонус 2)."

print("Примеры 3 раздела - assert пройдены.")


**Задание:**  
Напишите `def spell_save_dc(*, proficiency: int, ability_mod: int) -> int`,  
которая возвращает **Spell Save DC**: `8 + proficiency + ability_mod`.

Обращаем внимание на `*`

In [ ]:
# ВАШЕ РЕШЕНИЕ
def spell_save_dc(*, proficiency: int, ability_mod: int) -> int:
    # TODO: верните целое число DC
    return -1  # заглушка

# ТЕСТЫ
assert spell_save_dc(proficiency=3, ability_mod=4) == 15
assert spell_save_dc(ability_mod=5, proficiency=2) == 15
print("Задание 3 - assert пройдены.")


#### **Разбор/решение:**  


Keyword-only параметры снижают шанс перепутать порядок и типы аргументов.

In [ ]:
def spell_save_dc(*, proficiency: int, ability_mod: int) -> int:
    return 8 + proficiency + ability_mod

assert spell_save_dc(proficiency=3, ability_mod=4) == 15
assert spell_save_dc(ability_mod=5, proficiency=2) == 15
print("Разбор 3 — assert пройдены.")


## 4) `*args` и `**kwargs`: упаковка/распаковка

**Теория:**
- `*args` собирает дополнительные **позиционные** аргументы в кортеж.
- `**kwargs` собирает дополнительные **именованные** аргументы в словарь.
- Распаковка: `f(*seq)` и `g(**dict)` - удобно прокидывать параметры дальше.


In [ ]:
# Пример: суммарный урон с модификаторами
def total_damage(*hits: int, crit: bool = False, **mods):
    """Суммируем урон по ударам. Модификаторы (mods) добавляются в конце.
    Примеры mods: rage=2, hex=1, favored_foe=1
    """
    base = sum(hits)
    if crit:
        base *= 2
    extra = sum(mods.values()) if mods else 0
    return base + extra


assert total_damage(5, 7, 4, crit=True, rage=2, hex=1) == 31  # (5+7+4)*2 + (2+1)

hits = [3, 6, 2]
mods = {"bless": 1, "hunter_mark": 2}

assert total_damage(*hits, **mods) == 14  # 11 + 3
print("Примеры 4 раздела - assert пройдены.")


**Задание:**  
Реализуйте `def mean_initiative(*rolls, ignore_none: bool = True) -> float | None`,  
которая считает среднее значение инициативы (например, бросков `d20`).  
Если `ignore_none=True`, пропускайте `None`. Если после фильтрации нет значений — верните `None`.

Ниже — тесты `assert`.


In [ ]:
# ВАШЕ РЕШЕНИЕ
def mean_initiative(*rolls, ignore_none: bool = True):
    # TODO: посчитать среднее по броскам инициативы
    return None  # заглушка


# ТЕСТЫ
val1 = mean_initiative(12, 18, 7)
val2 = mean_initiative(20, None, 10)
val3 = mean_initiative(None, None, ignore_none=True)


assert isinstance(val1, (float, int)) and abs(val1 - (12+18+7)/3) < 1e-9
assert isinstance(val2, (float, int)) and abs(val2 - (20+10)/2) < 1e-9
assert val3 is None

print("Задание 4 - assert пройдены.")


#### **Разбор/решение:**  


Отфильтруйте входные данные и корректно обработайте пустой список.

In [ ]:
from typing import Optional


def mean_initiative(*rolls, ignore_none: bool = True) -> Optional[float]:
    values = []
    for r in rolls:
        if r is None and ignore_none:
            continue
        values.append(r)
    if not values:
        return None
    return sum(values) / len(values)


assert abs(mean_initiative(12, 18, 7) - (12+18+7)/3) < 1e-9
assert abs(mean_initiative(20, None, 10) - (20+10)/2) < 1e-9
assert mean_initiative(None, None, ignore_none=True) is None
print("Решение 4 - assert пройдены.")


## 5) Функции как объекты: передача/возврат, `lambda`, замыкания (`nonlocal`)



**Теория:**
- Функции - объекты класса: их можно передавать, хранить, возвращать.
- `lambda` - краткая форма объявления анонимной функции.
- Замыкание: внутренняя функция (определяем функцию внутри функции) "помнит" переменные внешней.
- `nonlocal` позволяет изменять переменную внешней функции.

`global` меняет переменную которую видят все функции и классы, а `nonlocal` - только переменную "родительской функции" которую видит только родительская и "дочерние" функции

**Зачем надо замыкание?**

- Хранить состояние без глобальных переменных и классов.

- Создавать “фабрики функций” - заранее подготавливать функцию с определёнными параметрами (Один из паттернов порграммирования).

- Инкапсулировать данные - держать какие-то значения скрытыми внутри функции, чтобы к ним нельзя было достучаться напрямую.

In [ ]:
def apply_effect(effect_func, value):
    """Применяет эффект (функцию) к значению, напр. к урону или AC."""
    return effect_func(value)

def bless(x):      # +1 к броскам (упрощённо)
    return x + 1

assert apply_effect(bless, 14) == 15

In [ ]:
def multiplier(factor: float):
    def mul(x: float) -> float:
        return x * factor
    return mul

double = multiplier(2.0)
assert double(7) == 14

In [ ]:
def make_inspiration_pool():
    points = 0
    def add(n=1):
        nonlocal points
        points += n
        return points
    return add

insp = make_inspiration_pool()
assert (insp(), insp(2), insp()) == (1, 3, 4)

print("Примеры 5 раздел - assert пройдены.")


**Задание:**  
1) Реализуйте `def compose(f, g)`, возвращающую новую функцию `h(x) = f(g(x))`.  
2) Используйте `multiplier(factor)` (множитель урона).  
3) Определите `crit(x)` — функция, удваивающая урон.  
4) Соберите `power_attack_then_crit = compose(crit, multiplier(2))` и проверьте на числе `10`.

Ниже — тесты `assert`.


In [ ]:
# ВАШЕ РЕШЕНИЕ
def compose(f, g):
    # TODO: вернуть функцию h(x) = f(g(x))
    return None  # заглушка

def multiplier(factor: float):
    # TODO: вернуть функцию, умножающую аргумент на factor
    return None  # заглушка

def crit(x: float) -> float:
    # TODO: удвоить урон (критический удар)
    return x  # заглушка

# ТЕСТЫ
try:
    power_attack_then_crit = compose(crit, multiplier(2))
    assert callable(power_attack_then_crit)
    assert power_attack_then_crit(10) == 40  # (10*2) -> 20 -> крит -> 40
    print("Задание 5 — assert пройдены (если вы это видите).")
except Exception as e:
    print("Задание 5 — тесты не пройдены:", e)


#### **Разбор/решение:**  


Композиция функций позволяет собирать эффекты в цепочку применений.

In [ ]:
# Пример решения (с assert)
def compose(f, g):
    def h(x):
        return f(g(x))
    return h

def multiplier(factor: float):
    def mul(x: float) -> float:
        return x * factor
    return mul

def crit(x: float) -> float:
    return x * 2

power_attack_then_crit = compose(crit, multiplier(2))
assert power_attack_then_crit(10) == 40
print("Решение 5 - assert пройдены.")